In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

tests_elapsed_path = "tests_elapsed_times.csv"
costs_path = "machine_costs.csv"  

## 1. Loading the data

In [ ]:
elapsed_times_df = pd.read_csv(tests_elapsed_path)